In [21]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import base64
import io
import psycopg2

import pandas as pd
import plotly.express as px

from moviepy.editor import VideoFileClip
import dash_table

In [55]:
app = dash.Dash()
df = pd.read_csv('./data/Iris.csv')
fig = px.scatter(df, x="SepalWidth", y="SepalLength", color="Species")

app.layout = html.Div([
    # Title
    html.Div(
        children=[
            html.H1('Video Analysis',
                    style={'textAlign': 'center'}),
        ]),

    # Upload Video
    html.Div(
        dcc.Upload(
            id='upload-video',
            children=html.Div([
                'Drag and Drop or Select a Video',
            ]),
            style={
                'width': '100%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            # Allow multiple files to be uploaded        
            multiple=True,
        ),
        style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center'}
    ),

    # Output Video
    html.Div(id='output-video',
             style={'display': 'flex', 
                    'justifyContent': 'space-around', 
                    'alignItems': 'center'}),
    
    html.Br(),
    html.Br(),

    # feature 1
    html.Div(id='feature1',
             children=[
                 html.H2('Face Detection And Mesh'),
                 dcc.Graph(id='graph1', 
                           figure=fig)
             ]),
    html.Br(),

    # feature 2
    html.Div(id='feature2',
             children=[
                 html.H2('Sentimental Analysis'),
                 dcc.Graph(id='graph2', 
                           figure=fig)
             ]),   

    html.Br(),

    # feature 3
    html.Div(id='feature3',
             children=[
                 html.H2('Pose Detection'),
                 dcc.Graph(id='graph3', 
                           figure=fig)
             ]),  

    html.Br(),

    # feature 4
    html.Div(id='feature4',
             children=[
                 html.H2('Face & Arm Detection'),
                 dcc.Graph(id='graph4', 
                           figure=fig)
             ]),   

    html.Br(),

    # feature 5
    html.Div(id='feature5',
             children=[
                 html.H2('Compare Text Summarization'),
                 dcc.Graph(id='graph5', 
                           figure=fig)
             ]),   

])


@app.callback(
    Output('output-video', 'children'),
    Input('upload-video', 'contents'),
    prevent_initial_call=True
)
def to_postgre(list_of_contents):
    if list_of_contents is not None:

        ########### About Database ###########
        # Connect to your postgres DB
        conn = psycopg2.connect("dbname=VideoWeb user=postgres password=1q2w3e")

        # Open a cursor to perform database operations
        cur = conn.cursor()

        # Create table if it doesn't exist
        cur.execute("CREATE TABLE IF NOT EXISTS video_metadata (id serial PRIMARY KEY, duration REAL, fps REAL, size TEXT, audio_fps REAL);")


        for contents in list_of_contents:
            # The contents are base64 encoded, so we must decode them
            decoded = base64.b64decode(contents.split(',')[1])

            # Write the binary data to a file
            with open('temp.mp4', 'wb') as f:
                f.write(decoded)

            # Read the video file and extract metadata
            clip = VideoFileClip('temp.mp4')

            # Extract metadata
            duration = clip.duration
            fps = clip.fps
            size = str(clip.size)
            audio_fps = clip.audio.fps if clip.audio is not None else None
            audio_nchannels = clip.audio.nchannels if clip.audio is not None else None
            audio_sample_width = clip.audio.sample_width if clip.audio is not None else None
            audio_sample_rate = clip.audio.sample_rate if clip.audio is not None else None

            # Insert the metadata into the database
            cur.execute("INSERT INTO video_metadata (duration, fps, size, audio_fps) VALUES (%s, %s, %s, %s);", (duration, fps, size, audio_fps))

        ########### About Table ###########
            # Create a DataFrame from the metadata
            df = pd.DataFrame({
                'Duration': [duration],
                'FPS': [fps],
                'Size': [size],
                'Audio FPS': [audio_fps],
                'Audio Channels': [audio_nchannels],
                'Audio Sample Width': [audio_sample_width],
                'Audio Sample Rate': [audio_sample_rate]
            
            })

            # Transpose the DataFrame
            df = df.transpose()
            df.columns = ['Value']            

            # Reset the index and rename the index column to 'Metadata'
            df = df.reset_index().rename(columns={'index': 'Metadata'})            

            # Create a DataTable from the DataFrame
            table = dash_table.DataTable(
                id='table',
                columns=[{"name": i, "id": i} for i in df.columns],
                data=df.to_dict('records'),
            )                  

        # Commit changes and close resources
        conn.commit()
        cur.close()
        conn.close()    

        children = [
            html.Div(
                children=[
                    html.Video(src=contents, 
                               controls=True, 
                               style={'width': '100%'})
                ],
                style={'width': '50%'}
            ),
            html.Div(
                children=[table],
                style={'width': '50%'}
            )
        ]
        return children

if __name__ == '__main__':
    app.run_server(debug=True)